In [1]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')

from lightning_train import DataModule, generate_trainer
from data import *
from model import *
from testing import *

from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *

import pandas as pd
from scipy.sparse import csr_matrix
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from functools import partial
import torchmetrics.functional as f
from torchmetrics import Metric
import torchmetrics 

In [2]:
data = an.read_h5ad('../data/pancreas/pancreas.h5ad', backed='r+')

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


Now that we know all the labels are the same, we can subset the samples we'd like to train on and train a model

In [3]:
common_labels = []
for i in range(4):
    labels = data.obs.reset_index(drop=True) # make index numeric 
    labels = labels[labels['batch'] == str(i)]
    labels.index.name = 'cell'
    common_labels.append(labels['celltype'].unique().categories)
    labels.to_csv(f'../data/pancreas/batch_{i}_labels.csv', index=True)

# Test refgenes to make sure this code is buggy .. 

In [4]:
train_cols = data.var.index
ref_cols = train_cols[1:]

Index(['A2M', 'ABAT', 'ABCA1', 'ABCA17P', 'ABCA7', 'ABCB6', 'ABCB7', 'ABCC5',
       'ABCC6', 'ABHD3',
       ...
       'VTRNA1-3', 'ZFP91-CNTF', 'SNORD10', 'LOC339290', 'ESF1', 'MIR663A',
       'LOC100379224', 'LOC100130093', 'LOC101928303', 'COPG'],
      dtype='object', name='index', length=2448)

In [10]:
datamodule = DataModule(
    datafiles=['../data/pancreas/pancreas.h5ad'],
    labelfiles=['../data/pancreas/labels.csv'],
    class_label='celltype',
    index_col='cell',
    assume_numeric_label=False,
#     currgenes=train_cols,
#     refgenes=ref_cols,
#     preprocess=True,
)

datamodule.prepare_data()
datamodule.setup()

model = SIMSClassifier(
    input_dim=datamodule.input_dim,
    output_dim=datamodule.num_labels,
)

assume_numeric_label=False, using sklearn.preprocessing.LabelEncoder and encoding target variables.
Transforming labelfile 1/1
Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights
Initializing network
Initializing explain matrix


/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [ ]:
wandb_logger = WandbLogger(
    project=f"Pancreas Model",
    name="Local, Default Settings"
)

trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=200,
)

trainer.fit(model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(

  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 167 K 
-----------------------------------
167 K     Trainable params
0         Non-trainable params
167 K     Total params
0.670     Total estimated 

Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:206: RuntimeWarning: invalid value encountered in true_divide
  precision = tp / (tp + fp)
/Users/julian/Documents/Projects/SIMS/notebooks/../src/model.py:207: RuntimeWarning: invalid value encountered in true_divide
  recall = tp / (tp + fn)
/Users/ju

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
trainer.test(model, datamodule=datamodule)

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:219: UserWarning: You called `self.log('test_confusion_matrix', ...)` in your `test_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Median f1 score is 0.9464114832535886 for epoch=199
Test f1 is [0.9771987  0.94736842 0.97843666 0.97135741 0.91       0.97109827
 0.98734177 0.         0.95348837 0.84210526 0.         0.94545455
 0.         0.        ]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_auroc': 0.5352076888084412,
 'test_balanced_accuracy': 0.9280354976654053,
 'test_confusion_matrix': 0.16276288032531738,
 'test_f1': 0.9635302424430847,
 'test_loss': 0.16146838665008545,
 'test_loss_epoch': 0.16146838665008545,
 'test_median_f1': 0.9464114904403687,
 'test_per_class_f1': nan,
 'test_per_class_precision': nan,
 'test_per_class_recall': nan,
 'test_precision': 0.9635302424430847,
 'test_recall': 0.9635302424430847,
 'test_specificity': 0.997194766998291,
 'test_total_accuracy': 0.9635302424430847,
 'test_weighted_accuracy': 0.9635302424430847}
--------------------------------------------------------------------------------


[{'test_loss': 0.16146838665008545,
  'test_loss_epoch': 0.16146838665008545,
  'test_total_accuracy': 0.9635302424430847,
  'test_balanced_accuracy': 0.9280354976654053,
  'test_weighted_accuracy': 0.9635302424430847,
  'test_precision': 0.9635302424430847,
  'test_recall': 0.9635302424430847,
  'test_f1': 0.9635302424430847,
  'test_per_class_f1': nan,
  'test_per_class_precision': nan,
  'test_per_class_recall': nan,
  'test_specificity': 0.997194766998291,
  'test_confusion_matrix': 0.16276288032531738,
  'test_auroc': 0.5352076888084412,
  'test_median_f1': 0.9464114904403687}]

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


In [12]:
t1 = data.obs[data.obs['batch'] == '0']
t2 = data.obs[data.obs['batch'] == '1']

In [13]:
t1['celltype'].unique().categories

Index(['MHC class II', 'PSC', 'acinar', 'activated_stellate', 'alpha', 'beta',
       'co-expression', 'delta', 'dropped', 'ductal', 'endothelial', 'epsilon',
       'gamma', 'macrophage', 'mast', 'mesenchymal', 'mesenchyme',
       'not applicable', 'quiescent_stellate', 'schwann', 't_cell',
       'unclassified', 'unclassified endocrine', 'unclear'],
      dtype='object')

wandb: Network error (ConnectionError), entering retry loop.


In [45]:
t2['celltype'].unique().categories == t1['celltype'].unique().categories

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])